In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
train=pd.read_csv('../input/tabular-playground-series-jan-2021/train.csv')

In [ ]:
train.head()

In [ ]:
test=pd.read_csv("../input/tabular-playground-series-jan-2021/test.csv")

In [ ]:
test.head()

In [ ]:
train.describe()

In [ ]:
train.info()

In [ ]:
#Let's try xgboost, one with gridsearchCV, and one without it.

In [ ]:
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

In [ ]:
xgbmodel=XGBRegressor()

In [ ]:
params_grid={"n_estimators": np.arange(10,160,50),
            "learning_rate": np.arange(0.05,0.35,0.1),
           }

In [ ]:
xgb_gs_cv=GridSearchCV(estimator=XGBRegressor(),param_grid=params_grid,
scoring='neg_mean_squared_error', cv=4, verbose=1)

In [ ]:
#forgot to split my data.

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x=train.drop(["target","id"],axis=1)
y=train["target"]

In [ ]:
xtrain,xtest,ytrain,ytest= train_test_split(x,y,test_size=0.25,random_state=42)

In [ ]:
xgbmodel.fit(xtrain,ytrain)

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
ypred = xgbmodel.predict(xtest)
xgbrmse = np.sqrt(mean_squared_error(ytest,ypred))

In [ ]:
xgbrmse

In [ ]:
#now let's fit it to the whole data before saving.
xgbmodel.fit(x,y)

In [ ]:
#saving the first model
import pickle
pickle.dump(xgbmodel, open("xgb.model", 'wb'))

In [ ]:
xgb_gs_cv.fit(xtrain,ytrain)

In [ ]:
gs_ypred = xgb_gs_cv.predict(xtest)
gs_rmse = np.sqrt(mean_squared_error(ytest,gs_ypred))

In [ ]:
gs_rmse

In [ ]:
xgb_gs_cv.fit(x,y)

In [ ]:
#saving the second model
pickle.dump(xgb_gs_cv, open("xgb_gs_cv.model", 'wb'))

In [ ]:
#If i don't set the seed i would get different results everytime i run the notebook.
from numpy.random import seed
seed(10) #arbitrarily

In [ ]:
#now let's build a neural network model.
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
tab_net= Sequential()

In [ ]:
n_cols=x.shape[1]
n_cols

In [ ]:
tab_net.add(Dense(250,activation='relu',input_shape=(n_cols,)))
tab_net.add(Dense(100,activation='relu'))
tab_net.add(Dense(25,activation='relu'))
tab_net.add(Dense(5,activation='relu'))
tab_net.add(Dense(1,activation='linear'))
tab_net.compile(optimizer='adam',loss='mean_squared_error')

In [ ]:
from keras.callbacks import EarlyStopping
early_stopping_monitor = EarlyStopping(patience=2)

In [ ]:
tab_net.fit(xtrain, ytrain, validation_split=0.15, epochs=30,
callbacks = [early_stopping_monitor])

In [ ]:
tab_net_ypred = tab_net.predict(xtest)
tab_net_rmse = np.sqrt(mean_squared_error(ytest,tab_net_ypred))

In [ ]:
tab_net_rmse

In [ ]:
#let's try a more patient one to see if the rmse gets any better.
early_stopping_monitor=EarlyStopping(patience=3)
tab_net.fit(xtrain, ytrain, validation_split=0.15, epochs=30,
callbacks = [early_stopping_monitor])

In [ ]:
#saving the model
tab_net.save('tab_net.h5')

In [ ]:
test=pd.read_csv("../input/tabular-playground-series-jan-2021/test.csv")

In [ ]:
xtest=pd.read_csv("../input/tabular-playground-series-jan-2021/test.csv").drop("id",axis=1)

In [ ]:
xtest.head()

In [ ]:
xgbmodel_sub=xgbmodel.predict(xtest)
xgb_gs_cv_sub=xgb_gs_cv.predict(xtest)
tab_net_sub=tab_net.predict(xtest)

In [ ]:
print(xgbmodel_sub.shape,
tab_net_sub.shape,
xgb_gs_cv_sub.shape)

In [ ]:
tab_net_sub=tab_net_sub.reshape((200000,))

In [ ]:
xgbmodel_sub=pd.DataFrame({"id":test.id, "target":xgbmodel_sub}).set_index("id")

In [ ]:
xgb_gs_cv_sub=pd.DataFrame({"id":test.id, "target":xgb_gs_cv_sub}).set_index("id")
tab_net_sub=pd.DataFrame({"id":test.id, "target":tab_net_sub}).set_index("id")

In [ ]:
xgbmodel_sub.to_csv("xgbmodel_sub.csv")
tab_net_sub.to_csv("tab_net_sub.csv")
xgb_gs_cv_sub.to_csv("xgb_gs_cv_sub.csv")

In [ ]:
#guess which one i submitted